In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split


In [2]:
# Load the dataset as a Pandas DataFrame
data = pd.read_csv('./datasets_aug.csv')
data.rename(columns = {'0':'text','1':'summary'}, inplace = True)
print(data.head())

# Define the maximum length of the input and output sequences
max_input_length = 150
max_output_length = 50

                                                text  \
0  ब्याङ्करहरूले भने तत्काल कर्जाको ब्याजदर एकल अ...   
1  हामीले पटक पटक गुहार्दा पनि सरकारले नसुनेकाले ...   
2  व्यवसायी मात्र होइन कतिपय राजनीतिक दलले पनि वर...   
3  कोभिड महामारीपछि थलिएको व्यापार व्यवसाय क्षेत्...   
4  कोभिडपछि थलिएका हाम्रो अहिले अवस्था निकै नाजुक...   

                                             summary  
0  ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन...  
1  ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन...  
2  ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन...  
3  ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन...  
4  ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन...  


In [35]:
model_path = "./model-300/"

def save_model(model, tokenizer, max_input_length, max_output_length):
    model.save(model_path+'seq2seq_model.h5')
    with open(model_path+'tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(model_path+'max_input_length.pickle', 'wb') as handle:
        pickle.dump(max_input_length, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(model_path+'max_output_length.pickle', 'wb') as handle:
        pickle.dump(max_output_length, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [14]:
model_path ="./model-300/"
def load_model():
    loaded_model = tf.keras.models.load_model(model_path+'seq2seq_model.h5')
    with open(model_path+'tokenizer.pickle', 'rb') as handle:
        loaded_tokenizer = pickle.load(handle)
    with open(model_path+'max_input_length.pickle', 'rb') as handle:
        loaded_max_input_length = pickle.load(handle)
    with open(model_path+'max_output_length.pickle', 'rb') as handle:
        loaded_max_output_length = pickle.load(handle)
    return loaded_model, loaded_tokenizer, loaded_max_input_length, loaded_max_output_length


In [17]:
# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 10000,oov_token='<OOV>')
tokenizer.fit_on_texts(data['text'])
input_sequences = tokenizer.texts_to_sequences(data['text'])
output_sequences = tokenizer.texts_to_sequences(data['summary'])

# Pad the input and output sequences to the same length
input_data = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_input_length, padding='post', truncating='post')
output_data = tf.keras.preprocessing.sequence.pad_sequences(output_sequences, maxlen=max_output_length, padding='post', truncating='post')


In [18]:
# Split the data into training and testing sets

# Write code with shuffle to split train test 


# train_data = input_data[:int(0.8 * len(input_data))]
# train_labels = output_data[:int(0.8 * len(input_data))]
# test_data = input_data[int(0.8 * len(input_data)):]
# test_labels = output_data[int(0.8 * len(input_data)):]
train_text,test_text,train_summ,test_summ = train_test_split(input_data,output_data,random_state=1,test_size=0.25, shuffle = True)



In [90]:
# Build the seq2seq model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.num_words + 1, 128, input_length=max_input_length, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.4,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.4,)),
    tf.keras.layers.RepeatVector(max_output_length),
    tf.keras.layers.LSTM(32,dropout=0.4,return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(tokenizer.num_words + 1, activation='softmax'))
])

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 128)          1280128   
                                                                 
 bidirectional_4 (Bidirectio  (None, 150, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 repeat_vector_2 (RepeatVect  (None, 50, 64)           0         
 or)                                                             
                                                                 
 lstm_8 (LSTM)               (None, 50, 32)            12416     
                                                      

# load the model



In [15]:
model, tokenizer, max_input_length, max_output_length = load_model()


In [33]:
# Compile the model

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')



In [34]:
# Train the model
with tf.device('GPU:0'):
    history = model.fit(train_text, train_summ, epochs=50, validation_data=(test_text, test_summ))



Epoch 1/50
1541/1541 [==============================] - 147s 88ms/step - loss: 1.9664 - val_loss: 2.6422
Epoch 2/50
1541/1541 [==============================] - 131s 85ms/step - loss: 1.9626 - val_loss: 2.6313
Epoch 3/50
1541/1541 [==============================] - 130s 84ms/step - loss: 1.9609 - val_loss: 2.6361
Epoch 4/50
1541/1541 [==============================] - 131s 85ms/step - loss: 1.9558 - val_loss: 2.6967
Epoch 5/50
1541/1541 [==============================] - 131s 85ms/step - loss: 1.9589 - val_loss: 2.6508
Epoch 6/50
1541/1541 [==============================] - 131s 85ms/step - loss: 1.9599 - val_loss: 2.6441
Epoch 7/50
1541/1541 [==============================] - 130s 84ms/step - loss: 1.9593 - val_loss: 2.6736
Epoch 8/50
1541/1541 [==============================] - 121s 79ms/step - loss: 1.9567 - val_loss: 2.6472
Epoch 9/50
1541/1541 [==============================] - 121s 78ms/step - loss: 1.9603 - val_loss: 2.6459
Epoch 10/50
1541/1541 [==============================] 

# save the model 

In [ ]:
# history.history.keys()
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [37]:
save_model(model, tokenizer, max_input_length, max_output_length)

In [10]:
def generate_summary(model, tokenizer, max_input_length, text):
    # Tokenize the input text
    sequence = tokenizer.texts_to_sequences([text])
    # Pad the sequence to the same length as the input sequences
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_input_length, padding='post', truncating='post')
    # Perform inference with the trained model
    prediction = model.predict(padded_sequence)[0]
    # Convert the predicted sequence back into text
    summary = tokenizer.sequences_to_texts([np.argmax(prediction, axis=-1)])[0]
    return summary


In [17]:
generate_summary(model, tokenizer, max_input_length, "अनुशासन एक त्यस्तो गुण हो जसद्वारा व्यक्तिले आफ्ना भावनाहरू र व्यवहारलाई नियन्त्रण गर्न सिक्छ। हाम्रो जीवनको हरेक मार्गमा अनुशासन अत्यन्त मूल्यवान छ। यसले व्यक्तिलाई जीवनमा प्रगति गर्न र सफलता प्राप्त गर्न प्रेरित गर्दछ। हामीले स्कूल, घर, कार्यालय, संस्था, कारखाना, खेल मैदान, रणभूमि वा अन्य स्थानमा अनुशासनको पालना गर्नुपर्दछ।अनुशासनले हामीलाई परिपक्व,सोच्न कार्य गर्न र जिम्मेवार निर्णयहरू लिन सक्षम गर्दछ। यसले हामीलाई आत्म-संचालित, आत्म-नियन्त्रित र आत्म-निर्देशित व्यक्ति बनाउँछ। यसले हामीलाई जिम्मेवार र प्रधान व्यक्ति बनाउँछ। अनुशासनलले  एक व्यक्तलाई सफल पार्नको लागि प्रेरित गर्न र उसको जीवनमा प्रगति गर्न ठूलो र महत्त्वपूर्ण भूमिका खेल्छ। अनुशासनले एक व्यक्तिको चरित्र निर्माण गर्दछ।  अनुशासन बिना, एक व्यक्तिको जीवन सुस्त र निष्क्रिय हुनेछ।अनुशासनको पलना गर्ने व्यक्तिले समाजमा उच्च पद र प्रतिष्ठा प्राप्त गर्दछ। अनुशासनको क्षेत्र ज्यादै फराकिलो पनि छ।घरपरिवारदेखि लिएर समाज, धर्म, संस्कृति, राजनीति, वित्त आदि सबै क्षेत्रमा अनुशासन चाहिन्छ । अनुशासन सामाजिक जीवनको एक अत्यावश्यक गुण हो। यो बिना, समाज मा जीवन अराजक र दयनीय हुन्छ। यसले चीजहरूलाई तपाईंको लागि चलाउन  सजिलो बनाउँदछ अन्तत: तपाईंको जीवनमा सफलता ल्याउँछ। अनुशासनको क्षेत्र ज्यादै फराकिलो पनि छ।घरपरिवारदेखि लिएर समाज, धर्म, संस्कृति, राजनीति, वित्त आदि सबै क्षेत्रमा अनुशासन चाहिन्छ। विद्यार्थीको जीवनमा अनुशासनको धेरैनै महत्त्व हुन्छ। विद्यार्थीलाई आकार दिने सबै गुणहरू मध्ये अनुशासन सबैभन्दा आधारभूत हुन्छ। अनुशासित विद्यार्थीलाई विद्यालयमा सबैले राम्रो नजरले हेर्छन। अनुशासनहीन बिद्यार्थीको पढाइ राम्रो हुन् सक्दैन भने अनुशासित बिद्यार्थीले पढाइमा अल्लेख्नीय सफलता हासिल गर्न सक्दछ।")


'<OOV> <OOV> <OOV> <OOV> परिवार दिँदा <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> पुगे कार्यक्रमका <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>'

In [16]:
text = "हामीले पटक पटक गुहार्दा पनि सरकारले नसुनेकाले आन्दोलनमा उत्रनु परेको हो, महासङ्घका अध्यक्ष कुमार कार्कीले भने। त्यस्तै व्यवसायीहरूले हाल धितो लिलामीका लागि दिने सूचना दिए पुग्ने प्रावधानलाई परिमार्जन गरी महिना पुर्याउनु पर्ने माग पनि राखेका छन्। त्यस्तै उक्त दलले समस्याग्रस्त उद्योग तथा व्यवसायले जरिवाना नतिरी कर तिर्ने सुविधा र भाखा नाघेका ऋणीले लिएको ऋणको किस्ता तथा ब्याज तिर्ने अवधि आगामी असार मसान्तसम्म दिनुपर्ने माग पनि गरेको छ। राष्ट्र ब्याङ्कका अधिकारीहरू चाहिँ तरलता अभाव, मुद्रास्फीति, भुक्तानी सन्तुलन तथा विदेशी मुद्रा सञ्चितिलगायतका कुराहरूलाई हेरेर ब्याजदर ह्वात्तै कम गर्न नसकिने बताउँछन्। राष्ट्र ब्याङ्कका डेपुटी गभर्नर बमबहादुर मिश्रले भने, अहिले हाम्रो भुक्तानी सन्तुलन, मुद्रास्फीति र विदेशी मुद्रा सञ्चितिमा सुधार आइरहेको छ।,ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन्दै कतिपय व्यवसायीहरूले आन्दोलन गरिरहेका छन् भने कतिपयले मन्त्रीहरू तथा प्रधानमन्त्रीलाई समेत भेटेर गुनासो गरिरहेका छन्। व्यवसायी मात्र होइन कतिपय राजनीतिक दलले पनि वर्तमान आर्थिक अवस्थाबारे सरकारको ध्यानाकर्षण गराएका छन्। मङ्गलवार प्रधानमन्त्री पुष्पकमल दाहाललाई भेटेर नेपाल कम्युनिस्ट पार्टी समाजवादीले अर्थतन्त्र सुधारको सुझावसहितको ज्ञापन पत्र बुझाएको छ। उक्त दलले बुझाएको ज्ञापन पत्रमा पनि अन्य सुधारका साथसाथै ब्याङ्क तथा वित्तीय क्षेत्रमा गर्नुपर्ने सुधारबारे पनि उल्लेख गरिएको छ। नेकपा समाजवादीले बुझाएको ज्ञापन पत्रमा भनिएको छ, ब्याङ्क तथा वित्तीय संस्थाहरूको बचत तथा ऋण नीतिमा तत्काल सुधार गरियोस्। ब्याङ्क तथा वित्तीय संस्थाहरूले निक्षेपमा बढीमा चार प्रतिशत ब्याज दिन पाउने र ऋण प्रवाहमा बढीमा सात प्रतिशत मात्रै ब्याज लिन पाउने व्यवस्था तत्काल लागु गरियोस्।बीबीसीसँग कुराकानी गरेका नेपाल राष्ट्र ब्याङ्ककाडेपुटी गभर्नर, ब्याङ्कर्स तथा पूर्व ब्याङ्कर्सले भने तत्कालै ब्याजदरलाई एकल अङ्कमा ल्याउन सम्भवनभएको बताए। त्यसलाई क्याश रिजर्भ रेसियो अर्थात् सीआरआर भनिन्छ र त्यो अहिले न्यूनतम चार प्रतिशत रहेको छ। अर्थमन्त्रालयले आश्वासन दिएर मात्र भएन मौद्रिक नीति सञ्चालन गर्ने राष्ट्र ब्याङ्कले त्यसलाई खासै वास्ता गरेको छैन, उनले भने। राष्ट्र ब्याङ्कले स्प्रेडदर अर्थात् ब्याङ्कहरूले लिने कर्जा र निक्षेपको ब्याजबीचको अन्तरलाई कम गर्नु भने पनि अझै ब्याङ्कहरूले त्यसलाई कम नगरेको विवरण आइरहेका छन्। तर ब्याङ्कहरूले त्यसका लागि राष्ट्र ब्याङ्कले दिएको समय अझै बाँकी रहेको बताएका छन्।,ब्याङ्कहरूले कर्जामा लिने ब्याजदर उच्च भएको भन्दै कतिपय व्यवसायीहरूले आन्दोलन गरिरहेका छन् भने कतिपयले मन्त्रीहरू तथा प्रधानमन्त्रीलाई समेत भेटेर गुनासो गरिरहेका छन्।"
generate_summary(model, tokenizer, max_input_length, text)

'ब्याङ्कहरूले ब्याजदर उत्पादित ब्याजदर कटौती <OOV> <OOV> आन्दोलन व्यवसायीहरूले आन्दोलन आन्दोलन लगाएपछि <OOV> कतिपयले मन्त्रीहरू <OOV> प्रधानमन्त्रीलाई पहल भेटेर गुनासो छन्। छन्। <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>'